# Download Any BAM File 🌕

Any BAM file will do. I just want to see what aligned data looks like in the HCA.

First I'll set up the DSS client.

In [ ]:
import hca, hca.dss, json
client = hca.dss.DSSClient()

****
#### Now I want to find a bundle that has a BAM file in it.

The `client` has a method `get_file` that sounds very promising, but I need to get a UUID. I don't know what RFC4122 is, but hopefully I won't have to.

In [ ]:
help(client.get_file)

****
#### Searching for bundles

The `post_search` method accepts a `query`, and there's even an example query in the data-store repo's [readme](https://github.com/HumanCellAtlas/data-store/blob/master/README.md). The function signature doesn't quite match the example, but it should be easy to fix. 

In [ ]:
client.post_search(replica="aws", es_query={
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "files.sample_json.donor.species": "Homo sapiens"
                }
            }, {
                "match": {
                    "files.assay_json.single_cell.method": "Fluidigm C1"
                }
            }, {
                "match": {
                    "files.sample_json.ncbi_biosample": "SAMN04303778"
                }
            }]
        }
    }
})

****
#### Well that didn't work!

But I can see how the results are structured. What if I just give it an empty query?

In [ ]:
search_response = client.post_search(replica="aws", es_query={})
search_response["total_hits"]

****
Okay great, many results. What does each result look like?

In [ ]:
search_response["results"][0]

____
Now I have an ID that I can work with, `bundle_fqid`! It's an FQID and not a UUID, and for a bundle, not a file. What happens if I provide the FQID to `get_bundle` as the UUID?

In [ ]:
try:
    client.get_bundle(uuid=search_response["results"][0]["bundle_fqid"], replica="aws")
    print("Completed successfully!")
except Exception as e:
    # If this operation fails, let's print the error (without raising the exception)
    print("Oh no! There was an error.")
    print(e)

****
Hmmm, a `DSSException` - it appears that it couldn't find a bundle with that UUID. This makes sense because FQIDs aren't UUIDs: the UUID is the part of the FQID before the first `.`. The timestamp is everything after. So, if I extract the UUID from the FQID, everything should work:

In [ ]:
bundle_uuid, bundle_version = search_response['results'][0]['bundle_fqid'].split('.', maxsplit=1)
client.get_bundle(uuid=bundle_uuid, replica="aws")

****
Nice! Now I can see the general structure of a bundle.

Using this information, what if I want to find a bundle with a BAM? I can write a function `find_bam()` that iterates over bundles until it finds a bundle with a BAM...

In [ ]:
def find_bam():
    for result in search_response["results"]:
        bundle_uuid, _ = results['bundle_fqid'].split('.', maxsplit=1)
        bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
        for file_dict in bundle_dict["bundle"]["files"]:
            if file_dict["name"].endswith(".bam"):
                return file_dict

Looks good! `find_bam()` will loop over each result in the `post_search` query we made earlier and GET each bundle until it finds a bundle containing a file that ends in `.bam`. One thing to remember is that sometimes, `post_search` can return a bundle that's been deleted but is lingering in the index, and that trying to GET that bundle could result in an error. It's no problem, as long as we can catch and ignore those cases...

In [ ]:
def find_bam():
    for result in search_response["results"]:
        bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
        try:
            bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
        except:
            continue
        for file_dict in bundle_dict["bundle"]["files"]:
            if file_dict["name"].endswith(".bam"):
                return file_dict

One last thing: each `post_search` request is paginated and only returns some 100 results per request. Luckily, there's a method I can use that will automatically and transparently paginate through all results, `post_search.iterate`.

In [ ]:
help(client.post_search)

In [ ]:
I can update my function `find_bam` to use it:

In [ ]:
def find_bam():
    result = client.post_search.iterate(replica="aws", es_query={})
    for result in results:
        bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
        try:
            bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
        except:
            continue
        for file_dict in bundle_dict["bundle"]["files"]:
            if file_dict["name"].endswith(".bam"):
                return file_dict

In [ ]:
find_bam()

Looks good! What if I want to look for another file type, like fastqs? I can generalize that code above...

In [ ]:
def find_ext(extension):
    result = client.post_search.iterate(replica="aws", es_query={})
    for result in results:
        bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
        try:
            bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
        except:
            continue
        for file_dict in bundle_dict["bundle"]["files"]:
            if file_dict["name"].endswith(extension):
                return file_dict

In [ ]:
fastq_file = find_ext('.fastq.gz')
print(f"Name: {fastq_file['name']}, UUID: {fastq_file['uuid']}")

If I want to download a file and know its UUID, I can use the `get_file` method...

In [ ]:
bam_file = client.get_file(uuid=file_dict['uuid'], replica="aws")
with open("Aligned.sortedByCoord.out.bam", "wb") as output_bam:
    output_bam.write(bam_file)

In [ ]:
import pysam
bam = pysam.AlignmentFile("Aligned.sortedByCoord.out.bam", "rb")
print(bam.header)